In [ ]:
# analyzer/gmail_scanner.py

import re
from base64 import urlsafe_b64decode
from email import message_from_bytes

# -----------------------------
# REGEX USADOS PARA DETECTAR DATOS SENSIBLES
# -----------------------------

# Teléfonos españoles comunes: 9 dígitos, comenzando por 6,7,8,9
REGEX_TELEFONO = r"\b[6789]\d{8}\b"

# Emails
REGEX_EMAIL = r"\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b"

# Cuenta bancaria:  ES + 22 dígitos (IBAN simplificado)
REGEX_CUENTA = r"\bES\d{22}\b"

# DNI español: 8 dígitos + letra
REGEX_DNI = r"\b\d{8}[A-Za-z]\b"

# NIE español: X/Y/Z + 7 dígitos + letra
REGEX_NIE = r"\b[XYZ]\d{7}[A-Za-z]\b"

# Nombres (si quieres algo básico)
REGEX_NOMBRE = r"\b[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+ [A-ZÁÉÍÓÚÑ][a-záéíóúñ]+\b"


# -------------------------------------------------------
# FUNCIÓN PRINCIPAL: escanear Gmail y detectar datos
# -------------------------------------------------------

def scan_gmail_for_sensitive_data(gmail_service, max_messages=50):
    """
    Escanea los últimos correos en Gmail y devuelve una lista con los datos sensibles detectados.
    """

    results = gmail_service.users().messages().list(
        userId="me",
        maxResults=max_messages
    ).execute()

    messages = results.get("messages", [])
    detecciones = []

    for msg in messages:
        msg_id = msg["id"]

        raw_msg = gmail_service.users().messages().get(
            userId="me",
            id=msg_id,
            format="raw"
        ).execute()

        # Decodificar mensaje
        raw_bytes = urlsafe_b64decode(raw_msg["raw"])
        email_message = message_from_bytes(raw_bytes)

        # Extraer texto de todas las partes
        cuerpo = _extract_email_text(email_message)

        # Detectar datos sensibles en el texto
        datos = _detectar_datos_sensibles(cuerpo)

        if any(datos.values()):
            detecciones.append({
                "message_id": msg_id,
                "snippet": raw_msg.get("snippet", ""),
                "detected": datos,
            })

    return detecciones


# -------------------------------------------------------
# EXTRAER TEXTO DEL CORREO
# -------------------------------------------------------

def _extract_email_text(email_message):

    parts = []

    if email_message.is_multipart():
        for part in email_message.walk():
            content_type = part.get_content_type()

            if content_type == "text/plain":
                try:
                    parts.append(part.get_payload(decode=True).decode("utf-8", errors="ignore"))
                except:
                    pass
    else:
        try:
            parts.append(email_message.get_payload(decode=True).decode("utf-8", errors="ignore"))
        except:
            pass

    return "\n".join(parts)


# -------------------------------------------------------
# DETECCIÓN DE DATOS SENSIBLES
# -------------------------------------------------------

def _detectar_datos_sensibles(texto):

    return {
        "telefonos": re.findall(REGEX_TELEFONO, texto),
        "emails": re.findall(REGEX_EMAIL, texto),
        "cuentas": re.findall(REGEX_CUENTA, texto),
        "dni": re.findall(REGEX_DNI, texto),
        "nie": re.findall(REGEX_NIE, texto),
        "nombres": re.findall(REGEX_NOMBRE, texto),
    }

